In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing

In [2]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"

films = pd.read_csv(film_csv, index_col=None, header=0)
display(films.head())

trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
display(trailers_meta.head())

,id,title,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties
0,60432,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д..."
1,64219,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес..."
2,65407,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл..."
3,75149,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""..."
4,75629,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...","{ ""Аудитория"" : [""для детей 13-16 лет"",""9 лет""..."


,Unnamed: 0,Unnamed: 0.1,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,1,4,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,2,5,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,3,6,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,4,7,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [ ]:
# WARNING: не запускать, т.к. скрипт уже был использован и испортит данные

trailers_meta['properties'].to_list()
cols = ["Место действия", "Качества", "Время действия", "На основе", "Аудитория", "Настроение", "Поджанр", "Про что", "Тематика"]

places = []
qualities = []
times = []
mains = []
audiences = []
moods = []
subgenres = []
abouts = []
themes = []

for i in range(0, len(trailers_meta)):
    trailers_info = json.loads(trailers_meta.iloc[i]['properties'])
    
    try:
        place = trailers_info['Место действия']
    except:
        place = ['не определено']
    places.append(place)
    
    try:
        quality = trailers_info['Качества']
    except:
        quality = ['не определено']
    qualities.append(quality)
    
    try:
        time = trailers_info['Время действия']
    except:
        time = ['не определено']
    times.append(time)
    
    try:
        main = trailers_info['На основе']
    except:
        main = ['не определено']
    mains.append(main)
    
    try:
        audience = trailers_info['Аудитория']
    except:
        audience = ['не определено']
    audiences.append(audience)
    
    try:
        mood = trailers_info['Настроение']
    except:
        mood = ['не определено']
    moods.append(mood)
    
    try:
        subgenre = trailers_info['Поджанр']
    except:
        subgenre = ['не определено']
    subgenres.append(subgenre)

    try:
        about = trailers_info['Про что']
    except:
        about = ['не определено']
    abouts.append(about)
    
    try:
        theme = trailers_info['Тематика']
    except:
        theme = ['не определено']
    themes.append(theme)
    

df = pd.DataFrame({
    'content_id': trailers_meta['content_id'],
    'Place of act': places,
    'Qualities': qualities,
    'Time of act': times,
    'Based on': mains,
    'Audience': audiences,
    'Mood': moods,
    'Subgenre': subgenres,
    'About': abouts,
    'Theme': themes
})

display(df.head())
display(trailers_meta.head())

trailers_df = pd.merge(trailers_meta, df, left_on='content_id', right_on='content_id', how='left')
trailers_df = trailers_df.drop_duplicates(subset=['content_id'])
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

trailers_df.to_csv(trailers_csv)

In [3]:
info = {}
min_len = 100
max_len = 0
del_array_name = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_meta)):
    name = trailers_meta.iloc[i]['trailers_name']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info_df = info_df.iloc[: , 3:]
        info[name] = info_df
    except:
        del_array_name.append(name)

In [5]:
print(del_array_name, len(del_array_name))

['0abceea32472c573ba7af504fd00fd', '2fa0ae09168251b817ab94cd60ce8a', 'fcbee265edc0503ad53a28c983c5e0', 'fde0e8cf8408f87356ac8fb722f481', 'f71302bf45d805cc80d4f8d5c41adc', 'a1c32f6263e9b6b26cdcbe208fa209', '194e610175ced32afa97ed202589a9', '1f01cfdb79b08ffde79eecd37760c1', '8bd3f4ebba62fa13ace809ba0efb68', '8d4386805fff1944baafba236650dd', '268fa5408e3080413d87557c7a5a40', '31eaaadbe52b0d65e819af4e4ae908', '1b988cc0438af387aeb16e8d1d36aa', '27d95cd4accb1f17508fcebce4bb3e', 'c5176f62d9e78b7508e91feed59e73', '32818249a83e2412ce506a131c43e0', '29882a40ff24e672596d99ff64ae17', '5c383cdc4f9194234194d3b3d807d3', '1cb6a48e086ab54867ea5ae89b1713', '967c11c3edbe5eaaed5b9bb2cd110c', '634679bb639198d034489b96eeea50', 'ab3bf566de4a72493b5d68ef2dc962', '921ce8b1f6a9ccc8aa0384d08c1522', '908293f3c6eb04489efc9ddc70b224', '0c9d3593f104130849949f5f384a4e', '6b1e914b7eaa6b0cc99285327db84e', '8e1f23602bfa26752e8a81c59d37b6', 'c1cec0a87f7c0128771c4f472724da', 'ac82f9a9ba4735c94f72471165f84b', '0e696eb09481

In [13]:
print(len(info))

912


"['21-й век', 'новейшее время']"

In [44]:
# использовать хорошие алгоритмы приближения значений!
# косинусная мера, евклидово расстояние, коэффициент Танимото
def make_session(df, start_ind = 0, session_len = 10):
    names = [df.iloc[start_ind]['trailers_name']]
    
    curr_info = info[names[0]]
    bright = np.mean(curr_info['brightness'].to_list())
    color = np.mean(curr_info['colorfulness'].to_list())
    tempo = np.mean(curr_info['tempo'].to_list())
        
    time_of_act = df.iloc[start_ind]['Time of act'] #array
    emotions = json.loads(df.iloc[start_ind]['emotions'].replace("\'", "\""))
    emotions_neu = emotions['neutral']
    emotions_neg = emotions['negative']
    emotions_pos = emotions['positive']
    emotions_skip = emotions['skip']
    emotions_speech = emotions['speech']
    genre = df.iloc[start_ind]['genres']
    mood = df.iloc[start_ind]['Mood']
    about = df.iloc[start_ind]['About']
    
    i = 0
    while i < len(df):
        if len(names) == session_len:
            break
            
        name = df.iloc[i]['trailers_name']
        if name not in names:
            i_emo = json.loads(df.iloc[i]['emotions'].replace("\'", "\""))
            i_genre = df.iloc[i]['genres']
            i_mood = df.iloc[i]['Mood']
            i_about = df.iloc[i]['About']
            i_time = df.iloc[i]['Time of act']
            if name not in del_array_name:
                i_curr_info = info[name]
                i_bright = np.mean(curr_info['brightness'].to_list())
                i_color = np.mean(curr_info['colorfulness'].to_list())
                i_tempo = np.mean(curr_info['tempo'].to_list())
                if (
                    np.linalg.norm(emotions_neu - i_emo['neutral']) < 0.04 and
                    np.linalg.norm(emotions_neg - i_emo['negative']) < 0.04 and
                    np.linalg.norm(emotions_pos - i_emo['positive']) < 0.04 and
                    len(list(set(i_genre) & set(genre))) > 0 and
                    len(list(set(i_mood) & set(mood))) > 0 and
                    len(list(set(i_about) & set(about))) > 0 and
                    len(list(set(i_time) & set(time_of_act))) > 0 and
                    np.linalg.norm(bright - i_bright) < 0.04 and
                    np.linalg.norm(color - i_color) < 0.04 and
                    np.linalg.norm(tempo - i_tempo) < 0.04
                ):
                    names.append(name)
                    genre = list(set(i_genre) | set(genre))
                    mood = list(set(i_mood) | set(mood))
                    about = list(set(i_about) | set(about))
                    time_of_act = list(set(i_time) | set(time_of_act))
                    emotions_neu = np.mean([emotions_neu, i_emo['neutral']])
                    emotions_neg = np.mean([emotions_neg, i_emo['negative']])
                    emotions_pos = np.mean([emotions_pos, i_emo['positive']])
                    emotions_skip = np.mean([emotions_skip, i_emo['skip']])
                    emotions_speech = np.mean([emotions_speech, i_emo['speech']])
                    bright = (bright + i_bright) / 2
                    color = (color + i_color) / 2
                    tempo = (tempo + i_tempo) / 2
                    i = 0
                else:
                    i += 1
            else:
                i += 1
        else:
            i += 1
        
    return names
    

sessions = []
j = 0
while j < len(trailers_meta):
    print(j)
    session = make_session(trailers_meta, j)
    if len(session) == 10:
        print("append", len(sessions), session)
        sessions.append(session)
    j += 1
    if len(sessions) > 50:
        break;
    
sess_df = pd.DataFrame(sessions)
display(sess_df.head())
sess_df.to_csv("/Volumes/Seagate/natasha-diploma/sessions.csv")

0
append 0 ['403a42b604f504069c41d39c5a4fc2', '23ee5640ec588ffaa78afa3268b04a', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd44ad5cc2d63441ebf437eccb5', '02fbcf93a730c9c9d5e65aef41c67e', '26b3f811a6b9ed75d3cc5198105fa4', '6fcff414343456465369b05f6851eb', 'e2f15819192cded77ed1825a129c0c', '41b772a5b18bc724562bfb60f4f3ac']
1
2
append 1 ['a5b2dd3ed41aabb5733c7cfffff0bc', '25e573b81ab69d6cf700a51281fc2a', '49c58f679f72e0f02b97bdeee1a1bb', 'c0be492c14bcec5ab3dfd34d95d6f5', 'b2274b9b4c8c4dc2f79d417ddd71e5', '7fd08607f52b89a753437eee177e32', '25d41576f3529994c878a5d15f6dc7', 'ece16e15b2286f8287e8935f12b6c2', '806c0ed32e135afb8db4f3aad71acf', 'f10b4adf1c732b6b260791fb24f430']
3
append 2 ['80870ec50515107ba7034ab758cebe', 'cd522afebcf91625c3cd811f3fb6e2', '061a59839837a123a916a70947f075', '07e873c5aac8ffaf97897fbd0dfbf6', '60fb638ff7ab469be8637111fdc837', 'd49c4a9b748b9c7abb9d98cdecebef', 'b0b062dc25c4dada316183941b100c', 'c7f809aab571e9acdf799a78c5ef61', '67c38de26

append 25 ['90785321743c3aacb1a93d1a6a63bc', '433b9cff26f4d11d4af6889dc0fba2', 'ae90166aa81ab2bf878dace61a5620', '8d3877a1b26b10642d125ec4fc4c9b', '8a2483e11828332a9e2fc8659ae38f', 'c739a34abff60f3677b0daf54cf077', 'b10d13f52983ef860b09db5e693373', 'b5cace1f414bee46d0441934d18bf9', '31f08ef4a727a8943644261454f1fa', '5ccf8f0bf03df5bf674c2aa36e7fe6']
31
append 26 ['29103e52e525684e0aea34ed67249e', '9fcec7f94c78ba3a1de4a9d1e5958d', '61c69f18631b3ea571150be66bafd8', 'b5a054300996482ae4355a41ea8c09', 'f9cada583c98a43c5c7220fffa3ab0', '403e3bd6b455beda1d25e77b27a927', 'cf0ea2b71dffa752f01f3365b2d052', '11d02b7fdc95ed52337e86d3c8787c', '38a1b2d1bb6603ab05014d67b59f5d', '4774caaf01df3d5d982e0cb77b1a13']
32
33
append 27 ['27cdcc4bbe3debe922e64c9fa90c55', '8c05cdd831c937a3982adb314741b0', '9aecf32a60e4e0f7f967caecb6c798', '209f1d1349a473f3eb0b285140c70a', '403a42b604f504069c41d39c5a4fc2', '23ee5640ec588ffaa78afa3268b04a', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd

append 49 ['67a82fb5d48c048069e746291df26b', '3c4ffbac90a3352b86b18a736bee9e', '4f687f43789873ca4541836ef2daf2', 'd6f4ce2516113cb955c447385ac3b5', '9e05015e2042286d7c9d21f214425e', '9d38d3b3c707d1a535bb21ca58c73a', '3eb635e807cc6a8484531b00486f71', '5bc1c8052fb210cff1aabade8dec0c', '49e57c1ad425edf22eed61b598c0ec', 'fcdebb544d991f71e3abe0c5b09af2']
60
61
62
append 50 ['5c58fd697b4262de0a85549a4b9151', '965709485ee74874de3063cf02ce35', '4e2a72ece58d87e675644865175631', 'ad9ad1a92ab6ca25c67507bf3d7c75', '772c5a2f027d06d8b7046a08374cdc', 'e82d0fb871a83d8983ef7a2fc99a37', '07e873c5aac8ffaf97897fbd0dfbf6', 'cd522afebcf91625c3cd811f3fb6e2', '061a59839837a123a916a70947f075', '44e4cff082f96ac13bd055263537f0']


,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3


In [ ]:
trailers_meta.to_csv(trailers_csv)

677


a1a39ec242c9d607fd9e9132c9e785
